# Sequence masking with PyTorch

Resources:
- Masking:
    - [Difference between `src_mask` and `src_key_padding_mask` in PyTorch Transformer layers (from StackOverflow)](https://stackoverflow.com/questions/62170439/difference-between-src-mask-and-src-key-padding-mask)
    - [UvA (University of Amsterdam) DL tutorial](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html)
    - [Judit Ács' blog post](https://juditacs.github.io/2018/12/27/masked-attention.html) (but watch out: **the attention matrix is not square!**)
- Masked language modeling:
    - [Kaggle notebook](https://www.kaggle.com/code/mojammel/masked-language-model-with-pytorch-transformer) (very similar to the PyTorch tutorial below)
    - [MLM with BERT blog post](https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c)
    - [PyTorch transformer tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)
    - [Tutorial with TensorFlow](https://keras.io/examples/nlp/masked_language_modeling/#create-bert-model-pretraining-model-for-masked-language-modeling) (this is actually a good reference, with tensor shapes etc.)
    - [Tutorial with PyTorch](https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial)

In [39]:
import sys
import torch

sys.path.append('../../modules/')

from models import TransformerClassifier, FFNN

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

Generate a vocabulary.

In [2]:
# Vocabulary size.
q = 8

vocab = torch.arange(q).to(dtype=torch.int64)
mask_idx = vocab.max() + 1

# Enalarge the vocabulary with the special tokens.
vocab = torch.hstack([vocab, torch.Tensor(mask_idx).to(dtype=torch.int64)])

vocab

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

Generate sequences of tokens.

**Note:** in the case considered all sequences have the same length and therefore no padding is (ever) needed.

In [3]:
# Tree depth.
l = 8

batch_size = 64
seq_len = 2 ** l

sequences = torch.randint(q, (batch_size, seq_len))

sequences

tensor([[7, 1, 6,  ..., 1, 5, 5],
        [4, 7, 0,  ..., 7, 7, 4],
        [1, 0, 2,  ..., 5, 5, 3],
        ...,
        [1, 4, 3,  ..., 4, 4, 4],
        [6, 2, 3,  ..., 3, 5, 3],
        [1, 1, 0,  ..., 1, 3, 3]])

## Building attention masks

Given the sequences, generate trainable input embeddings for them (just the semantic part, we're skipping the positional encoding here as it's not essential).

In [4]:
hidden_dim = 128

embedding_layer = torch.nn.Embedding(
    num_embeddings=vocab.shape[0],
    embedding_dim=hidden_dim
)

input_sequence_embeddings = embedding_layer(sequences)

input_sequence_embeddings.shape

torch.Size([64, 256, 128])

Instantiate a transformer Encoder model.

**Note:** by convention, we stick with having the batch dimension as the first one.

In [9]:
# A single encoder layer to be used in the full stack.
encoder_layer = torch.nn.TransformerEncoderLayer(
    d_model=hidden_dim,
    nhead=1,
    dim_feedforward=2048,
    batch_first=True
)

# Stack of encoder layers.
transformer_encoder = torch.nn.TransformerEncoder(
    encoder_layer,
    num_layers=1
)

encoder_output = transformer_encoder(input_sequence_embeddings)

encoder_output.shape

/home/moscato/miniconda3/envs/tree-language/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([64, 256, 128])

Generate masked sequences from the original one to perform masked language modeling (MLM): every token in every sequence is converted to the masked one (set conventionally) with some probability.

**Mask:** we pass the mask as the encoder's `src_key_padding_mask` option, which means it should have shape `(batch_size, seq_len)` and (if of boolean type) contain `False` when no masking is needed and `True` when it is. In practice, it's obtained by simply checking the masked sequences against the padding value.

In [10]:
masked_sequences = sequences.clone()

masking_rate = 0.1

for i in range(sequences.shape[0]):
    for j in range(sequences.shape[1]):
        if torch.rand(1) < masking_rate:
            masked_sequences[i, j] = mask_idx

mask = (masked_sequences == mask_idx)

**Questions:**
- Should we pass the masked sequences or the original ones (always along with the mask) to the encoder?

Answer: we should pass the masked sequences to the encoder, and then use the decoder to generate logits for every token in every sequence and compare this with the ground truth (non-masked sequences) via the cross-entropy loss.

What to do with the **masked embeddings**?
 
 - If we pass the masked sequences, we should have input embeddings be created for the `<mask>` token, so it should be explicitly modeled. Could we indicate it as a padding token as we instantiate the `Embedding` layer for the input embeddings?
 
A: the `<mask>` token should be explicitly modeled as part of the vocabulary. It shouldn't be among the possible **predicted** tokens though.

- If we pass the original sequences we're leaving the original embeddings for the masked tokens, is the mask sufficient to tell the model to ignore them?

A: masking is not sufficient. Indeed, we should pass the masked sequences to the encoder.

- What about the gradient? Should we "disconnect" the masked token from the compute graph?

A: still not clear. The attention mask should avoid connecting the embeddings of the masked tokens with the loss, but it's just a guess (and what about residual connections?).

What about the **full model** (with a decoder as well)?

- How do we tell which are the masked tokens to predict for in the sequences? The decoder won't know anything about the masking, so it'll have no way to distinguish between a masked token and a not masked one.

A: it's true that the decoder won't know explicitly which are the masked tokens, but indirectly it will because we pass the masked sequences as input and because in the end we'll probably have to select ony the loss terms correspnding to the predictions for the masked tokens.

- Should the decoder predict for one masked token at the time (how to select them? Mask them one by one?) or all masked tokens together (in which case, see the previous point)?

A: the decoder should predict for all the tokens, masked and non-masked, so that for an input of shape `(batch_size, seq_len)` we get a final output of shape `(batch_size, seq_len, vocab_size)`, the last dimension corresponding to the logits over the vocabulary (restricted to the token that should be predicted, e.g. not `<mask>`). This means that we'll effectively predict for the entire sequence of tokens every time and computing the cross-entropy loss with the non-masked sequences we'll get a tensor of loss values of shape `(batch_size, seq_len)`. Now we can choose (not clear which is the right choice though!) whether to use compute the final loss as the mean over all the entries of this tensor or just over the ones corresponding to the masked tokens (by applying the mask over the tensor of loss values).

In [11]:
encoder_output_masked = transformer_encoder(
    embedding_layer(masked_sequences),
    src_key_padding_mask=mask
)

encoder_output_masked

tensor([[[-0.2322,  0.0388,  1.6767,  ...,  0.0242,  1.3980,  1.6808],
         [-0.8812,  0.0447,  0.0232,  ..., -0.4531,  0.2737,  0.0878],
         [-1.5290,  0.0563,  0.0552,  ..., -0.2086,  0.0685, -0.2435],
         ...,
         [-0.4059, -0.8452, -0.8674,  ...,  0.4784,  0.0504,  1.4562],
         [-0.3940,  0.8562, -0.2764,  ..., -0.2095,  0.3218,  0.1295],
         [-0.0765,  0.9645, -0.4538,  ..., -0.2111,  0.3759,  0.1533]],

        [[-0.4931, -1.5078, -0.5607,  ..., -1.6448,  1.0646,  0.1621],
         [-0.2812,  0.0954,  1.4611,  ...,  0.0152,  1.1096,  1.7686],
         [-0.7402, -1.2070,  0.0637,  ...,  0.4589, -0.2991,  1.0881],
         ...,
         [-0.4373,  0.1567,  1.2907,  ...,  0.0658,  1.3477,  1.7567],
         [-0.1572,  0.3443,  1.5482,  ..., -0.1596,  1.2010,  1.7385],
         [-0.3564, -1.5720, -0.8014,  ..., -1.6988,  0.9743,  0.0201]],

        [[-0.6714, -0.9081, -0.8615,  ...,  0.3797, -0.1105,  1.4537],
         [-1.2523, -0.8404,  0.0987,  ...,  0

In [12]:
encoder_output_masked.shape

torch.Size([64, 256, 128])

The final output layer maps each token in each sequence to a set of logits (or probabilities) over the "proper" vocabulary (excluding special tokens), so it has tensors of shape `(batch_size, seq_len, hidden_dim)` as input and outputs tensors of shape `(batch_size, seq_len, vocab_size)`.

In [13]:
output_layer = torch.nn.Linear(
    in_features=hidden_dim,
    out_features=q
)

output_logits = output_layer(encoder_output_masked)
output_probs = torch.nn.Softmax(dim=-1)(output_logits)

output_logits.shape, output_probs.shape, output_probs.sum(dim=-1)

(torch.Size([64, 256, 8]),
 torch.Size([64, 256, 8]),
 tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
        grad_fn=<SumBackward1>))

Observation on the loss function (**should masking be considered at this stage?**):
- We need the model (output layer) to output the predicted logits for each token in each sequence, i.e. a tensor of shape `(batch_size, seq_len, vocab_size)`, where the last dimension represents the logits over the vocabulary.
- PyTorch's `CrossEntropyLoss` function accepts the logits and the true labels as the input, with the latter either as they are (class labels) or with one-hot encoding. In this case, if the predicted logits are put in the shape PyTorch expects (see point below), no one-hot encoding is needed for the targets.
- Without any aggregation, we should have a value for the loss for each token in each sequence, which gives a tensor of loss values of shape `(batch_size, seq_len)`. PyTorch assumes that the predicted logits have shape `(batch_size, n_classes, [additional dims])`, so the last two dimensions of the output logits need to be switched.
- **Guess:** probably the final loss should be computed only for the masked tokens, so we have to apply the mask to the loss tensor before aggregating the values.

In [14]:
loss_fn = torch.nn.CrossEntropyLoss(
    reduction='none'  # Default: 'mean'.
)

loss_tensor = loss_fn(
    torch.permute(output_logits, dims=(0, 2, 1)),
    sequences
)

loss_tensor, loss_tensor.shape

(tensor([[1.7630, 1.8346, 2.2416,  ..., 2.3638, 2.4226, 2.6556],
         [2.3062, 1.6842, 2.1129,  ..., 1.6795, 1.6789, 2.3856],
         [2.4108, 1.9434, 1.3742,  ..., 2.6980, 2.6533, 2.3493],
         ...,
         [2.4751, 2.4196, 1.3885,  ..., 2.3911, 2.4065, 2.3443],
         [1.9154, 1.4418, 2.4791,  ..., 2.2590, 2.6034, 2.3484],
         [2.4772, 2.3990, 2.0787,  ..., 2.3824, 2.3456, 1.4215]],
        grad_fn=<ViewBackward0>),
 torch.Size([64, 256]))

In [15]:
# Use masking if predicting only for the masked tokens,
# drop the mask (!) to predict for the whole sequence.
loss = loss_tensor[mask].mean()

loss

tensor(2.2145, grad_fn=<MeanBackward0>)

## Building a model for MLM

In [40]:
model = TransformerClassifier(
    seq_len=seq_len,
    embedding_size=hidden_dim,
    n_tranformer_layers=1,
    n_heads=1,
    vocab_size=vocab.shape[0],
    n_special_tokens=1,
    embedding_agg=None,
    decoder_hidden_sizes=[],
    decoder_activation='identity',
    decoder_output_activation='identity'  # 'identity' --> Output logits
)

# Output shape: (batch_size, seq_len, vocab_size) (excluding
# the special tokens from the vocabulary, which shouldn't be
# predicted).
model(masked_sequences, src_key_padding_mask=mask)

tensor([[[-5.6495e-01,  4.1420e-01,  7.3308e-01,  ...,  1.1685e-01,
          -7.8136e-01, -7.0507e-02],
         [-1.0830e+00,  5.0556e-01,  1.5749e+00,  ...,  5.5534e-01,
          -7.4043e-01,  7.9937e-02],
         [-3.5241e-01,  6.9179e-01,  1.8018e+00,  ...,  4.5706e-01,
          -9.7880e-01, -4.2974e-01],
         ...,
         [-3.0115e-01,  2.1086e-01,  1.1576e+00,  ...,  3.0247e-01,
           1.2638e-01, -6.9550e-01],
         [-2.0102e-01,  7.2382e-01,  2.0122e-01,  ...,  4.9295e-02,
           3.1154e-02, -4.5668e-01],
         [-2.9357e-01,  5.9051e-01,  3.9578e-02,  ...,  5.6005e-02,
           2.1187e-01,  2.4956e-01]],

        [[-2.9578e-01, -1.5329e-01,  1.0964e+00,  ...,  6.1371e-01,
          -1.2509e-01,  7.1609e-01],
         [-6.5517e-02,  1.1782e-01,  1.0875e+00,  ...,  1.0212e-01,
          -1.0336e+00,  3.1837e-02],
         [-7.4655e-01,  8.2542e-01,  6.1313e-01,  ...,  6.9905e-01,
          -5.5864e-01, -5.0904e-01],
         ...,
         [-5.2889e-01,  1

In [41]:
loss_fn = torch.nn.CrossEntropyLoss(
    reduction='none'
)

# Use masking if predicting only for the masked tokens,
# drop the mask (!) to predict for the whole sequence.
loss = loss_fn(
    torch.permute(model(masked_sequences, src_key_padding_mask=mask), (0, 2, 1)),
    sequences
)[mask].mean()

loss

tensor(2.2933, grad_fn=<MeanBackward0>)